In [107]:
import pandas as pd
import numpy as np
import json

In [108]:
provs = ['BC', 'AB', 'SK', 'MB', 'ON', 'QC', 'NB', 'NS', 'PE', 'NL', 'YT', 'NT', 'NU']
parties = ['Bloc', 'Conservative', 'Green', 'Liberal', 'NDP']

In [109]:
def get_df_sums(df, party=None, prov=None):
    q = df.copy()
    if party:
        q = q[q['party'] == party]
    if prov:
        q = q[q['prov_abbr'] == prov]
    return len(q)

def create_summary_df(df, voting_system):
    tot_df = pd.DataFrame(columns=parties + ['region', 'voting_system'], index=range(14))
    pct_df = pd.DataFrame(columns=parties + ['region', 'voting_system'], index=range(14))

    tot_df['region'] = provs + ['Canada']
    pct_df['region'] = provs + ['Canada']

    tot_df['voting_system'] = [voting_system]*len(tot_df)
    pct_df['voting_system'] = [voting_system]*len(pct_df)
    
    for i in pct_df.index:
        for p in parties:
            
            if pct_df.region[i] == 'Canada':
                n_seats_party = get_df_sums(df, party=p)
                tot_df.set_value(i, p, n_seats_party)
                pct_df.set_value(i, p, n_seats_party*1./len(df))

            else:
                n_seats_party = get_df_sums(df, party=p, prov=pct_df.region[i])
                n_seats_prov = get_df_sums(df, prov=pct_df.region[i])
                pct_df.set_value(i, p, n_seats_party*1./n_seats_prov)
                tot_df.set_value(i, p, n_seats_party)

    return tot_df, pct_df

## Popular Vote results

In [110]:
pop = pd.DataFrame.from_csv('../data/popular_vote/table_tableau09.csv', index_col=None)

In [111]:
pop.columns = ['party', 'NL', 'PE', 'NS', 'NB', 'QC', 'ON', 'MB', 'SK', 'AB', 'BC', 'YT', 'NT', 'NU', 'Canada']

In [112]:
party_row_dict = {'Bloc': 2, 'Conservative': 7, 'Green': 10, 'Liberal': 11, 'NDP': 15}

In [113]:
pop2 = pd.DataFrame(columns=parties + ['region', 'voting_system'], index=range(14))
pop2['region'] = provs + ['Canada']
pop2['voting_system'] = ['POP']*len(pop2)

In [114]:
for i in pop2.index:
    for p in parties:
        pop2.set_value(i, p, pop[pop2.region[i]][party_row_dict[p]])

In [115]:
pop2['Other'] = 100. - (pop2['Bloc'] + pop2['Conservative'] + pop2['Green'] + pop2['Liberal'] + pop2['NDP'])

In [116]:
print pop2['Other'][6]
pop2.set_value(6, 'Other', 0)

-0.1


,Bloc,Conservative,Green,Liberal,NDP,region,voting_system,Other
0,0,29.9,8.2,35.1,26,BC,POP,0.8
1,0,59.6,2.5,24.5,11.6,AB,POP,1.8
2,0,48.5,2.1,23.9,25.1,SK,POP,0.4
3,0,37.4,3.2,44.7,13.6,MB,POP,1.1
4,0,35.1,2.8,44.8,16.6,ON,POP,0.7
5,19.4,16.7,2.2,35.7,25.4,QC,POP,0.6
6,0,25.4,4.7,51.6,18.4,NB,POP,0
7,0,17.9,3.4,62,16.3,NS,POP,0.4
8,0,19.3,6,58.3,16,PE,POP,0.4
9,0,10.3,1.1,64.5,21.1,NL,POP,3


## FPTP results

In [117]:
fptp = pd.DataFrame.from_csv('../data/fptp/summary_results_table11.csv', index_col=None)

In [118]:
fptp['candidate_and_party'] = fptp['Elected Candidate/Candidat \xc3\xa9lu']
fptp['riding_name'] = fptp['Electoral District Name/Nom de circonscription']
fptp['riding_number'] = fptp['Electoral District Number/Num\xc3\xa9ro de circonscription']
fptp['prov'] = fptp[fptp.columns[0]]

In [119]:
fptp = fptp[['prov', 'candidate_and_party', 'riding_name', 'riding_number']].copy()

In [120]:
party_dict = {'Liberal/Lib\xc3\xa9ral': 'Liberal',
           'Conservative/Conservateur': 'Conservative',
           'NDP-New Democratic Party/NPD-Nouveau Parti d\xc3\xa9mocratique': 'NDP',
           'Bloc Qu\xc3\xa9b\xc3\xa9cois/Bloc Qu\xc3\xa9b\xc3\xa9cois': 'Bloc',
           'Green Party/Parti Vert': 'Green'}

In [121]:
fptp['party'] = pd.Series(index=fptp.index, dtype=str)

In [122]:
for i in fptp.index:
    for k in party_dict:
        if k in fptp['candidate_and_party'][i]:
            fptp.set_value(i, 'party', party_dict[k])
            

In [123]:
fptp[fptp.party != fptp.party]

,prov,candidate_and_party,riding_name,riding_number,party


In [124]:
del fptp['candidate_and_party']

In [125]:
prov_dict = {'Newfoundland and Labrador/Terre-Neuve-et-Labrador': 'NL',
             'Prince Edward Island/Île-du-Prince-Édouard': 'PE',
             'Nova Scotia/Nouvelle-Écosse': 'NS',
             'New Brunswick/Nouveau-Brunswick': 'NB',
             'British Columbia/Colombie-Britannique': 'BC',
             'Yukon': 'YT',
             'Northwest Territories/Territoires du Nord-Ouest': 'NT',
             'Nunavut': 'NU',
             'Alberta': 'AB',
             'Manitoba': 'MB',
             'Ontario': 'ON',
             'Quebec/Qu\xc3\xa9bec': 'QC',
             'Saskatchewan': 'SK'
            }

In [126]:
fptp['prov_abbr'] = fptp['prov'].map(prov_dict)

In [127]:
fptp[fptp.prov_abbr != fptp.prov_abbr]

,prov,riding_name,riding_number,party,prov_abbr


In [128]:
del fptp['prov']

In [129]:
fptp['region_number'] = fptp['prov_abbr'].map(lambda x: provs.index(x))

In [130]:
fptp['seat_type'] = ['local']*len(fptp)

In [131]:
fptp.to_csv('../web/data/fptp_clean.csv', index=None)

In [132]:
tot_df_fptp, pct_df_fptp = create_summary_df(fptp, 'FPTP')
#  fptp2 = create_summary_df(fptp, 'FPTP')

## DMP Simulation Results

In [133]:
dmp = pd.DataFrame.from_csv('../data/dmp/dmp_2_5_0_reformatted.csv', index_col=None)

In [134]:
prov_dict_2 = {'Newfoundland': 'NL',
             'Prince Edward Island': 'PE',
             'Nova Scotia': 'NS',
             'New Brunswick': 'NB',
             'British Columbia': 'BC',
             'Yukon': 'YT',
             'Northwest Territories': 'NT',
             'Nunavut': 'NU',
             'Alberta': 'AB',
             'Manitoba': 'MB',
             'Ontario': 'ON',
             'Quebec': 'QC',
             'Saskatchewan': 'SK'
            }

In [135]:
dmp['prov_abbr'] = dmp['prov'].map(prov_dict_2)
del dmp['prov']

In [136]:
dmp['riding_number'] = range(len(dmp))

In [137]:
# I need 1 row per seat
dmp_1 = dmp[['riding_name', 'riding_number', 'party_1', 'prov_abbr']].copy()
dmp_2 = dmp[['riding_name', 'riding_number', 'party_2', 'prov_abbr']].copy()
dmp_1['party'] = dmp_1['party_1']
dmp_2['party'] = dmp_2['party_2']
del dmp_1['party_1']
del dmp_2['party_2']

In [138]:
dmp_2.index = range(len(dmp_1), len(dmp_1) + len(dmp_2))

In [139]:
dmp_3 = pd.concat([dmp_1, dmp_2])

In [140]:
for i in dmp_3.index:
    if dmp_3['party'][i] == 'Bloc Quebecois':
        dmp_3.set_value(i, 'party', 'Bloc')

In [141]:
dmp_3['region_number'] = dmp_3['prov_abbr'].map(lambda x: provs.index(x))

In [142]:
dmp_3['seat_type'] = ['local']*len(dmp_3)

In [143]:
dmp_3.to_csv('../web/data/dmp_clean.csv', index=None)

In [144]:
tot_df_dmp, pct_df_dmp = create_summary_df(dmp_3, 'DMP')

## MMP Simulation Results

Wilf Day's blog: http://wilfday.blogspot.ca/

My simulation using this model lets voters elect 209 local MP in the ten provinces, and 126 regional MPs in 42 regions.

In Atlantic Canada each province is one region. The other regions are:

Ontario:
Ottawa—Cornwall 10, East Central Ontario (Kingston—Peterborough) 9, Durham—Rouge Park 6, York Region 10, Scarborough—Don Valley 8, Toronto Central 8, North York—Etobicoke 8, Brampton—Mississauga North 7, Mississauga—Halton 8, Hamilton—Niagara—Brant 11, Central Ontario (Barrie—Owen Sound) 6, Waterloo-Wellington-Dufferin 8, London-Oxford-Perth-Huron 7, Windsor—Sarnia 6, Northern Ontario 9.

Quebec:
Montreal-est 8, Montreal-ouest 6, Montreal-nord—Laval 8, Laurentides—Lanaudière 9, Outaouais—Abitibi-Témiscamingue—Nord 6, Longueuil—Suroît 10, Montérégie-est—Estrie 6, Mauricie—Centre-du-Québec 6, Quebec City—Saguenay-Lac-Saint-Jean—Côte-Nord 11, Chaudière-Appalaches—Bas-Saint-Laurent—Gaspésie 8.

Manitoba:
Winnipeg 8, Manitoba North and South 6

Saskatchewan:
Regina—South Saskatchewan 6, Saskatoon—North Saskatchewan 8

Alberta:
Calgary 10, South & Central Alberta 8, Edmonton 10, Northern Alberta 6

British Columbia:
Vancouver-Richmond-Delta 9, Burnaby—North Shore—Coquitlam—Maple Ridge 8, Surrey—Fraser Valley 9, BC Interior and North 9, Vancouver Island 7

Voters for all three parties would elect either local or regional MPs in each of the 42 regions across Canada, except no Conservative in 8-MP Montreal-est and in 6-MP Outaouais—Abitibi-Témiscamingue—Nord.  Bloc voters would elect MPs in each of Quebec’s regions except 6-MP Montreal West. 

The results for 338 MPs are: 145 Liberals, 106 Conservatives, 70 NDP, 14 Bloc, and 3 Greens.



Lib, Con, NDP, Green, Bloc
B.C.                15, 13, 11, 3
Alberta           8, 21, 4, 1       
Sask.              3, 7, 4
Manitoba       6, 5, 2, 1
Ontario           55, 43, 20, 3
Quebec          28, 13, 20, 2, 15
N.B.                5, 3, 2
P.E.I.               2, 1, 1
N.S.                7, 2, 2
Nfld & Lab.    5, 1, 1
Territories      3, 0, 0
Total               137, 109, 67, 10, 15

In [146]:
# mmp = pd.DataFrame.from_csv('../data/mmp/mmp_14_reformatted.csv', index_col=None)
mmp = pd.DataFrame.from_csv('../data/mmp/mmp_12_reformatted.csv', index_col=None)

In [147]:
mmp = mmp.replace(to_replace=' ', value=np.nan)

In [148]:
mmp.head()

,Electoral District,prov_abbr,n_local,n_region,BQ,C,Gr,L,ND,In,BQ.1,C.1,Gr.1,L.1,NDP
0,Vancouver Island,BC,5,2,NaN,NaN,1.0,NaN,4.0,NaN,NaN,1.0,NaN,1.0,NaN
1,BC Interior and North,BC,6,3,NaN,3.0,NaN,1.0,2.0,NaN,NaN,NaN,NaN,2.0,1.0
2,Surrey—Richmond—Abbotsford—Langley,BC,7,5,NaN,3.0,NaN,4.0,NaN,NaN,NaN,1.0,1.0,1.0,2.0
3,Vancouver—Burnaby—North Shore—Maple Ridge,BC,9,5,NaN,NaN,NaN,6.0,3.0,NaN,NaN,3.0,1.0,NaN,1.0
4,Metropolitan Edmonton,AB,7,4,NaN,5.0,NaN,1.0,1.0,NaN,NaN,1.0,NaN,2.0,1.0


In [149]:
mmp['region_number'] = range(len(mmp))

In [150]:
mmp_local = mmp[list(mmp.columns[4:9]) + ['region_number', 'prov_abbr']].copy()
mmp_regional = mmp[list(mmp.columns[10:]) + ['prov_abbr']].copy()

In [151]:
mmp_regional.columns = parties + ['region_number', 'prov_abbr']
mmp_local.columns = parties + ['region_number', 'prov_abbr']

In [152]:
mmp_local['seat_type'] = ['local']*len(mmp_local)
mmp_regional['seat_type'] = ['regional']*len(mmp_regional)

In [153]:
mmp_regional.index = range(len(mmp_local), len(mmp_local)+len(mmp_regional))
mmp2 = pd.concat([mmp_local, mmp_regional])

In [154]:
mmp3 = pd.DataFrame(columns=['region_number', 'prov_abbr', 'seat_type', 'party', 'riding_number'], index=range(338))
i2 = 0
n = 0
for i1 in mmp2.index:
    for party in parties:
        if mmp2[party][i1] > 0:
            for _ in range(int(mmp2[party][i1])):
                for col in ['region_number', 'prov_abbr', 'seat_type']:
                    mmp3.set_value(i2, col, mmp2[col][i1])
                if mmp2['seat_type'][i1] == 'local':
                    mmp3.set_value(i2, 'riding_number', n)
                    n += 1
                else:
                    mmp3.set_value(i2, 'riding_number', -1)

                mmp3.set_value(i2, 'party', party)
                i2 += 1

In [156]:
mmp3.to_csv('../web/data/mmp_clean.csv', index=None)

In [157]:
tot_df_mmp, pct_df_mmp = create_summary_df(mmp3, 'MMP')

In [178]:
mmp.head()

,Electoral District,prov_abbr,n_local,n_region,BQ,C,Gr,L,ND,In,BQ.1,C.1,Gr.1,L.1,NDP,region_number
0,Vancouver Island,BC,5,2,NaN,NaN,1.0,NaN,4.0,NaN,NaN,1.0,NaN,1.0,NaN,0
1,BC Interior and North,BC,6,3,NaN,3.0,NaN,1.0,2.0,NaN,NaN,NaN,NaN,2.0,1.0,1
2,Surrey—Richmond—Abbotsford—Langley,BC,7,5,NaN,3.0,NaN,4.0,NaN,NaN,NaN,1.0,1.0,1.0,2.0,2
3,Vancouver—Burnaby—North Shore—Maple Ridge,BC,9,5,NaN,NaN,NaN,6.0,3.0,NaN,NaN,3.0,1.0,NaN,1.0,3
4,Metropolitan Edmonton,AB,7,4,NaN,5.0,NaN,1.0,1.0,NaN,NaN,1.0,NaN,2.0,1.0,4


In [179]:
len(mmp)

35

In [186]:
(mmp.n_region + mmp.n_local).mean()

9.742857142857142

In [187]:
(mmp.n_region + mmp.n_local)

0      7
1      9
2     12
3     14
4     11
5     12
6     11
7     14
8      6
9      8
10     9
11    13
12    13
13    12
14    15
15    13
16    12
17    15
18     9
19    10
20    11
21     7
22    11
23     6
24    12
25    13
26     6
27    12
28    10
29    11
30     4
31     7
32     2
33     2
34     2
dtype: int64

## Instant Runoff Voting (IRV/AV)

In [158]:
irv = pd.DataFrame.from_csv('../data/irv/irv_prov_grenier.csv', index_col=None)

In [159]:
irv

,prov_abbr,Liberal,Conservative,NDP,Green,Bloc
0,BC,26,2,13,1,0
1,AB,6,26,2,0,0
2,SK,2,8,4,0,0
3,MB,8,4,2,0,0
4,ON,96,16,9,0,0
5,QC,51,5,20,0,2
6,NB,10,0,0,0,0
7,NS,11,0,0,0,0
8,PE,4,0,0,0,0
9,NL,7,0,0,0,0


In [160]:
parties = ['Bloc', 'Conservative', 'Green', 'Liberal', 'NDP']
irv2 = pd.DataFrame(columns=['prov_abbr', 'party'], index=range(338))
i2 = 0
for i1 in irv.index:
    for party in parties:
        if irv[party][i1] > 0:
            for _ in range(int(irv[party][i1])):
                irv2.set_value(i2, 'prov_abbr', irv['prov_abbr'][i1])
                irv2.set_value(i2, 'party', party)
                i2 += 1

In [161]:
irv2['riding_number'] = irv2.index

In [162]:
irv2['region_number'] = irv2['prov_abbr'].map(lambda x: provs.index(x))

In [163]:
irv2['seat_type'] = ['local']*len(irv2)

In [164]:
irv2.to_csv('../web/data/irv_clean.csv', index=None)

In [165]:
irv3 = irv.copy()
for p in parties:
    irv3.set_value(13, p, irv3[p].sum())
irv3.set_value(13, 'prov_abbr', 'Canada')

,prov_abbr,Liberal,Conservative,NDP,Green,Bloc
0,BC,26.0,2.0,13.0,1.0,0.0
1,AB,6.0,26.0,2.0,0.0,0.0
2,SK,2.0,8.0,4.0,0.0,0.0
3,MB,8.0,4.0,2.0,0.0,0.0
4,ON,96.0,16.0,9.0,0.0,0.0
5,QC,51.0,5.0,20.0,0.0,2.0
6,NB,10.0,0.0,0.0,0.0,0.0
7,NS,11.0,0.0,0.0,0.0,0.0
8,PE,4.0,0.0,0.0,0.0,0.0
9,NL,7.0,0.0,0.0,0.0,0.0


In [166]:
irv3['region'] = irv3['prov_abbr']
irv3['voting_system'] = ['IRV']*len(irv3)
del irv3['prov_abbr']

In [167]:
irv3['total_seats'] = irv3.sum(axis=1)

In [168]:
irv3[irv3['region'] == 'BC'][parties].sum()

Bloc             0.0
Conservative     2.0
Green            1.0
Liberal         26.0
NDP             13.0
dtype: float64

In [169]:
pct_df_irv = irv3.copy()
for p in parties:
    pct_df_irv[p] = irv3[p]/irv3['total_seats']

In [170]:
tot_df_irv = irv3[irv3.columns[:-1]].copy()

## Combined Results for bar chart

In [171]:
pct_df_concat = pd.concat([pop2, pct_df_fptp, pct_df_mmp, pct_df_irv, pct_df_dmp])
tot_df_concat = pd.concat([tot_df_fptp, tot_df_mmp, tot_df_irv, tot_df_dmp])

pct_df_concat.index = range(len(pct_df_concat))
tot_df_concat.index = range(len(tot_df_concat))

In [172]:
def expand_df(df_concat):
    df_all = pd.DataFrame(columns=['party', 'region', 'voting_system', 'value'], index=range(len(df_concat)*5))
    i2 = 0
    for i1 in df_concat.index:
        if df_concat['voting_system'][i1] == 'POP':
            add_cols = parties + ['Other']
        else:
            add_cols = parties

        for party in add_cols:
            df_all.set_value(i2, 'party', party)
            df_all.set_value(i2, 'region', df_concat['region'][i1])
            df_all.set_value(i2, 'voting_system', df_concat['voting_system'][i1])
            if df_concat['voting_system'][i1] == 'POP':
                df_all.set_value(i2, 'value', df_concat[party][i1]*0.01)
            else:
                df_all.set_value(i2, 'value', df_concat[party][i1])
            i2 += 1
    return df_all

In [173]:
pct_df_all = expand_df(pct_df_concat)
tot_df_all = expand_df(tot_df_concat)

In [174]:
pct_df_all.to_csv('../web/data/pct_summary.csv', index=None)
tot_df_all.to_csv('../web/data/tot_summary.csv', index=None)

In [176]:
obj = {}
for region in list(set(tot_df_all.region)):
    obj[region] = {}
    for system in list(set(tot_df_all.voting_system)):
        obj[region][system] = {}
        for party in list(set(tot_df_all.party)):
            row = tot_df_all.query("region == @region and party == @party and voting_system == @system")
            obj[region][system][party] = list(row.value)[0]
        obj[region][system]['Other'] = 0

In [177]:
json.dump(obj, open('../web/data/tot_summary.json', 'w'))

In [188]:
obj

{'AB': {'DMP': {'Bloc': 0,
   'Conservative': 22,
   'Green': 0,
   'Liberal': 8,
   'NDP': 4,
   'Other': 0},
  'FPTP': {'Bloc': 0,
   'Conservative': 29,
   'Green': 0,
   'Liberal': 4,
   'NDP': 1,
   'Other': 0},
  'IRV': {'Bloc': 0.0,
   'Conservative': 26.0,
   'Green': 0.0,
   'Liberal': 6.0,
   'NDP': 2.0,
   'Other': 0},
  'MMP': {'Bloc': 0,
   'Conservative': 21,
   'Green': 0,
   'Liberal': 9,
   'NDP': 4,
   'Other': 0}},
 'BC': {'DMP': {'Bloc': 0,
   'Conservative': 13,
   'Green': 3,
   'Liberal': 15,
   'NDP': 11,
   'Other': 0},
  'FPTP': {'Bloc': 0,
   'Conservative': 10,
   'Green': 1,
   'Liberal': 17,
   'NDP': 14,
   'Other': 0},
  'IRV': {'Bloc': 0.0,
   'Conservative': 2.0,
   'Green': 1.0,
   'Liberal': 26.0,
   'NDP': 13.0,
   'Other': 0},
  'MMP': {'Bloc': 0,
   'Conservative': 11,
   'Green': 3,
   'Liberal': 15,
   'NDP': 13,
   'Other': 0}},
 'Canada': {'DMP': {'Bloc': 15,
   'Conservative': 111,
   'Green': 4,
   'Liberal': 140,
   'NDP': 72,
   'Other': 0